In [1]:
from dotenv import load_dotenv

load_dotenv("../.envrc")

True

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)

for i, embedding in enumerate(embeddings):
    print(embedding)


[-0.019187437370419502, -0.03813096508383751, -0.031003428623080254, -0.004613928031176329, -0.03536667302250862, -0.004004158079624176, 0.012920353561639786, 0.05103099346160889, -0.005826693493872881, -0.03720953315496445, -0.010813258588314056, -0.00223582424223423, 0.027249954640865326, -0.0022663127165287733, 0.005884282756596804, 0.0340387299656868, -0.016545098274946213, -0.010088309645652771, -0.03170805051922798, 0.07658714056015015, 0.059947188943624496, -0.018713170662522316, 0.0029590793419629335, 0.0189299788326025, 0.03975702077150345, 0.045719217509031296, 0.020786389708518982, 0.0065618050284683704, 0.013381068594753742, -0.004803634248673916, 0.029973594471812248, -0.022344691678881645, 0.006992031820118427, -0.023997846990823746, -0.01504099927842617, -0.0035671559162437916, -0.007554375566542149, 0.018157603219151497, -0.009715672582387924, -0.042494211345911026, 0.012147977948188782, -0.01872672140598297, 0.02227693982422352, -0.0015904840547591448, -0.0307595208287

In [9]:
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#임베딩 모델 API 호출
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

#PDF 문서 로드
loader = PyPDFium2Loader("../data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

#PDF 문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_documents(pages)

#OpenAI 임베딩 모델로 청크들을 임베딩 변환하기
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])

print(len(embeddings), len(embeddings[0]))

for i, embedding in enumerate(embeddings):
    print(embedding)

/Users/craig/workspace/git_repo/langchain-rag-practice/.venv/lib/python3.13/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


51 1536
[-0.04760844260454178, -0.019360488280653954, 0.003317152615636587, 0.005685057025402784, 0.043894898146390915, -0.03563331067562103, 0.0018541631288826466, 0.07664917409420013, -0.018213044852018356, -0.006389169488102198, 0.013216453604400158, -0.03536209836602211, -0.002790372120216489, -0.0007934305467642844, 0.020528793334960938, 0.031377341598272324, -0.04038998484611511, -0.011474426835775375, -0.03596711531281471, 0.030814053490757942, 0.00286078336648643, -0.015959884971380234, -0.01796269416809082, -0.017576735466718674, -0.029958685860037804, 0.001666399766691029, -0.006331797689199448, 0.01082768663764, 0.006404816638678312, -0.008423272520303726, 0.042017266154289246, -0.028748655691742897, 0.015271419659256935, -0.05774766206741333, -0.014144839718937874, -0.0070150475949049, -0.004782749805599451, 0.044395603239536285, 0.020747851580381393, 0.01770191267132759, 0.00747924018651247, -0.007693082094192505, -0.02288626693189144, -0.051989585161209106, 0.016783958300

In [14]:
documents = [
    "안녕하세요",
    "제 이름은 홍두깨입니다.",
    "이름이 무엇인가요?",
    "랭체인은 유용합니다.",
]
examples= embeddings_model.embed_documents(documents)

#예시 질문과 답변 임베딩
embedded_query_q = embeddings_model.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = embeddings_model.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

In [16]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

print(cos_sim(embedded_query_q, embedded_query_a))
for i, example in enumerate(examples):
    print(f"이 대화에서 언급된 이름은 무엇입니까? <-> {documents[i]}")
    print(cos_sim(embedded_query_q, example))

for i, example in enumerate(examples):
    print(f"이 대화에서 언급된 이름은 홍길동입니다. <-> {documents[i]}")
    print(cos_sim(embedded_query_a, example))

0.6491181289078307
이 대화에서 언급된 이름은 무엇입니까? <-> 안녕하세요
0.17845794170394286
이 대화에서 언급된 이름은 무엇입니까? <-> 제 이름은 홍두깨입니다.
0.2889011471041724
이 대화에서 언급된 이름은 무엇입니까? <-> 이름이 무엇인가요?
0.5001776024141352
이 대화에서 언급된 이름은 무엇입니까? <-> 랭체인은 유용합니다.
0.06867948637667336
이 대화에서 언급된 이름은 홍길동입니다. <-> 안녕하세요
0.20121476330890034
이 대화에서 언급된 이름은 홍길동입니다. <-> 제 이름은 홍두깨입니다.
0.4701907930911285
이 대화에서 언급된 이름은 홍길동입니다. <-> 이름이 무엇인가요?
0.28952925177148514
이 대화에서 언급된 이름은 홍길동입니다. <-> 랭체인은 유용합니다.
0.14843950996648797


In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

documents = [
    "안녕하세요",
    "제 이름은 홍두깨입니다.",
    "이름이 무엇인가요?",
    "랭체인은 유용합니다.",
]
examples = ko_embedding.embed_documents(documents)

embedded_query_q = ko_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = ko_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
for i, example in enumerate(examples):
    print(f"이 대화에서 언급된 이름은 무엇입니까? <-> {documents[i]}")
    print(cos_sim(embedded_query_q, example))

for i, example in enumerate(examples):
    print(f"이 대화에서 언급된 이름은 홍길동입니다. <-> {documents[i]}")
    print(cos_sim(embedded_query_a, example))

0.6070006202883593
이 대화에서 언급된 이름은 무엇입니까? <-> 안녕하세요
0.489547865375518
이 대화에서 언급된 이름은 무엇입니까? <-> 제 이름은 홍두깨입니다.
0.2947340874992772
이 대화에서 언급된 이름은 무엇입니까? <-> 이름이 무엇인가요?
0.7486293553870536
이 대화에서 언급된 이름은 무엇입니까? <-> 랭체인은 유용합니다.
0.2757840690697951
이 대화에서 언급된 이름은 홍길동입니다. <-> 안녕하세요
0.350474393799522
이 대화에서 언급된 이름은 홍길동입니다. <-> 제 이름은 홍두깨입니다.
0.35903605568114694
이 대화에서 언급된 이름은 홍길동입니다. <-> 이름이 무엇인가요?
0.4962498557924057
이 대화에서 언급된 이름은 홍길동입니다. <-> 랭체인은 유용합니다.
0.2546046807555932


In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
bge_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

documents = [
    "안녕하세요",
    "제 이름은 홍두깨입니다.",
    "이름이 무엇인가요?",
    "랭체인은 유용합니다.",
]
examples = bge_embedding.embed_documents(documents)

embedded_query_q = bge_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = bge_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
for i, example in enumerate(examples):
    print(f"이 대화에서 언급된 이름은 무엇입니까? <-> {documents[i]}")
    print(cos_sim(embedded_query_q, example))

for i, example in enumerate(examples):
    print(f"이 대화에서 언급된 이름은 홍길동입니다. <-> {documents[i]}")
    print(cos_sim(embedded_query_a, example))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.9554540817196477
이 대화에서 언급된 이름은 무엇입니까? <-> 안녕하세요
0.9224834837585106
이 대화에서 언급된 이름은 무엇입니까? <-> 제 이름은 홍두깨입니다.
0.9431682165682777
이 대화에서 언급된 이름은 무엇입니까? <-> 이름이 무엇인가요?
0.9360429120265862
이 대화에서 언급된 이름은 무엇입니까? <-> 랭체인은 유용합니다.
0.8853416825412523
이 대화에서 언급된 이름은 홍길동입니다. <-> 안녕하세요
0.9212055583489451
이 대화에서 언급된 이름은 홍길동입니다. <-> 제 이름은 홍두깨입니다.
0.9322696161981421
이 대화에서 언급된 이름은 홍길동입니다. <-> 이름이 무엇인가요?
0.9006508994465532
이 대화에서 언급된 이름은 홍길동입니다. <-> 랭체인은 유용합니다.
0.9105264291475871
